## MLflow's Model Registry

In [1]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

In [4]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.list_experiments()

AttributeError: 'MlflowClient' object has no attribute 'list_experiments'

In [5]:
client.create_experiment(name="my-cool-experiment")

'2'

Let's check the latest versions for the experiment with id `1`...

In [8]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [9]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: f1a9718eaf7044de83768e8ece85deb4, rmse: 6.3024
run id: 15319e28ce844b9d80ba3352f16b86ac, rmse: 6.3086
run id: 12a1a911140443b7aa45d7a251b27429, rmse: 6.4490
run id: 0ee083f3f2464e06a03a2af07d5c108d, rmse: 6.6095
run id: 41026863d0e94709b73e14e6c1f7e3bb, rmse: 6.7423


### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.

In [30]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [31]:
run_id = "41026863d0e94709b73e14e6c1f7e3bb"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="Taxi-models")

Registered model 'Taxi-models' already exists. Creating a new version of this model...
2023/05/25 22:53:29 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: Taxi-models, version 4
Created version '4' of model 'Taxi-models'.


<ModelVersion: aliases=[], creation_timestamp=1685051609449, current_stage='None', description=None, last_updated_timestamp=1685051609449, name='Taxi-models', run_id='41026863d0e94709b73e14e6c1f7e3bb', run_link=None, source='/home/godwin/Workflow/mlops-zoomcamp/02-experiment-tracking/mlruns/1/41026863d0e94709b73e14e6c1f7e3bb/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [32]:
model_name = "Taxi-models"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 4, stage: None
version: 2, stage: Staging


In [40]:
model_version = 4
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: aliases=[], creation_timestamp=1685051609449, current_stage='Staging', description=None, last_updated_timestamp=1685051746615, name='Taxi-models', run_id='41026863d0e94709b73e14e6c1f7e3bb', run_link=None, source='/home/godwin/Workflow/mlops-zoomcamp/02-experiment-tracking/mlruns/1/41026863d0e94709b73e14e6c1f7e3bb/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [16]:
client.delete_registered_model('nyc-taxi-regressor')

In [42]:
from datetime import datetime

date = datetime.today().date() 
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1684960878217, current_stage='Production', description='The model version 2 was transitioned to Production on 2023-05-25', last_updated_timestamp=1685051818723, name='Taxi-models', run_id='f1a9718eaf7044de83768e8ece85deb4', run_link=None, source='/home/godwin/Workflow/mlops-zoomcamp/02-experiment-tracking/mlruns/1/f1a9718eaf7044de83768e8ece85deb4/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [22]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df



def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [33]:
df = read_dataframe("data/green_tripdata_2021-01.parquet")

In [34]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

/tmp/ipykernel_7379/2852634549.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.download_artifacts`` is deprecated since 2.0. This method will be removed in a future release. Use ``mlflow.artifacts.download_artifacts`` instead.
  client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')


'/home/godwin/Workflow/mlops-zoomcamp/02-experiment-tracking/preprocessor'

In [35]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [36]:
X_test = preprocess(df, dv)

In [37]:
target = "duration"
y_test = df[target].values

In [45]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

2023/05/25 23:07:28 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.3.2, required: mlflow==2.3)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


[23:07:29] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: user 1min 17s, sys: 414 ms, total: 1min 17s
Wall time: 20.1 s


{'rmse': 4.29692488630116}

In [44]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

2023/05/25 23:06:02 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.3.2, required: mlflow==2.3)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


CPU times: user 1.09 s, sys: 667 ms, total: 1.76 s
Wall time: 1.93 s


{'rmse': 5.878874721516509}

In [43]:
client.transition_model_version_stage(
    name=model_name,
    version=4,
    stage="Production",
    archive_existing_versions=True
)

<ModelVersion: creation_timestamp=1652971637398, current_stage='Production', description='The model version 4 was transitioned to Staging on 2022-05-19', last_updated_timestamp=1652972763255, name='nyc-taxi-regressor', run_id='b8904012c84343b5bf8ee72aa8f0f402', run_link=None, source='./mlruns/1/b8904012c84343b5bf8ee72aa8f0f402/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>